In [10]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a0917377-5d7a-406b-8db1-402ac4123a69-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

In [12]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("rag_langchain")

LangSmith 추적을 시작합니다.
[프로젝트명]
rag_langchain


In [14]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000401730?sid=101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000401730?sid=101'}, page_content='\n"물가 오르는데 월급은 최저임금?"…\'공무원\' 꿈꾸는 MZ 29%→23%\n\n\n   지난 5월, 공무원 취직을 꿈꾸는 청년세대(15~29세)의 비율이 일 년 새 6%p 넘게 줄어든 것으로 나타났습니다.    통계청이 오늘(16일) 발표한 \'2024년 5월 경제활동인구조사 청년층 부가조사 결과\'에 따르면 청년층 취업 시험 준비자는 56만 5천명으로 집계됐습니다.    \n\n\n\n청년층 취업시험 준비분야별 비율 (자료 제공=통계청)    이 중 일반직 공무원 시험을 준비하는 비율은 23.2%(13만 1천명)로 일 년 전 29.3%와 비교해 6.2%p(5만 5천명) 가량 감소했습니다.    공직 사회에선 최저 임금 수준에 불과한 급여와 부족한 사회적 처우 때문에 공무원에 대한 청년층의 선호도가 갈수록 낮아지고 있다고 지적합니다.    전국공무원노동조합(이하 전공노)에 따르면 올해 9급 1호봉은 본봉과 직급 보조비, 급식비 등을 합쳐 세전 222만 2천원 수준입니다. 박중배 전공노 대변인은 "실수령액은 180~190만원으로 최저임금에 가깝다"며 "생활이 어려운 금액"이라고 설명했습니다.    또한 "사기업에서는 흔한 자녀 학자금 지원, 퇴직금 같은 복지도 없어 젊은 직원들에겐 \'메리트가 없는 직장\'으로 여겨진다"며 6급 이하 실무직 공무원들에 대한 임금 대책이 필요하다고 강조했습니다.    반면 일반 사기업과 전문직에 대한 청년층의 선호도는 높아지고 있는 것으로 나타났습니다. 일반 기업체를 준비하는 청년은 29.7%(16만 8천명)로 일 년 전에 비해 2.4% 증가했습니다. 일 년 새 기능 분야 자격증 취업준비 비율은 2.7%, 고시 및 전문직을 준비하는 청년층도 1.5% 증가했습니다.    올해 5월, 청년층이 취업하기까진 평균 11.5개월이 걸리는 걸로 조사됐

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

2

In [17]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [18]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [19]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
class StreamChain:
    def __init__(self, chain):
        self.chain = chain

    def stream(self, query):
        response = self.chain.stream(query)
        complete_response = ""
        for token in response:
            print(token, end="", flush=True)
            complete_response += token
        return complete_response


# 생성자에 chain 을 매개변수로 전달하여 chain 객체를 생성합니다.
chain = StreamChain(rag_chain)

In [21]:
answer = chain.stream("사기업과 전문직에 대한 청년층의 선호도를 알려주세요")

일반 사기업과 전문직에 대한 청년층의 선호도는 높아지고 있습니다. 일반 기업체를 준비하는 청년은 29.7%로 일 년 전에 비해 2.4% 증가했으며, 기능 분야 자격증 취업준비 비율은 2.7%, 고시 및 전문직을 준비하는 청년층도 1.5% 증가했습니다.

In [22]:
answer = chain.stream("고시생의 합격률은 얼마입니까")

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.

In [23]:
answer = chain.stream("공직사회의 급여는 어느정도 수준입니까")

공직사회의 급여는 9급 1호봉 기준으로 본봉과 직급 보조비, 급식비 등을 합쳐 세전 222만 2천원 수준이며, 실수령액은 180~190만원으로 최저임금에 가깝습니다.